<br>    
<img src="https://uniandes.edu.co/sites/default/files/logo-uniandes.png" width=200 height="100" align="left">
<h1 style="text-align:center;">IELE 4922</h1>
<h3 style='text-align: right;'> Departamento de Ingeniería Eléctrica y Electrónica &emsp;<br> Facultad de Ingeniería &emsp; <br>
     Universidad de los Andes &emsp; <br>
     Taller # 2 &emsp; <br>
</h3>
&emsp;<b>Integrante 1:</b> Mauricio Ricardo Delgado Quintero - 201712801
<br>
&emsp;<b>Integrante 2:</b> Juan Camilo Pico Garrido - 201731674
<br>

# Punto 3


(*The Lazy Adaptable Lion* [<span style="color:red">Puterman, 2014,</span> Ch. 3])

Un león requiere comer 6kg de carne por día y puede almacenar en su estomago hasta 30kg. Esto significa que con el estomago lleno, puede sobrevivir 5 días sin comer. Las cebras tienen una biomasa comestible promedio de 164kgm lo suficiente para cubrir las necesidades alimenticias de varios leones.
<img src="leones.png" width=400>
Si el león caza en grupo, las probabilidades de tener una caza exitosa aumentan con el número de integrantes del grupo. Cada vez que el león sale de caza, esto le representa un consumo energético de 0.5kg. Por observación se tiene que las probabilidades de una caza exitosa son:

$$p(1)=0.10+c(0.01) \quad p(2)=0.27-(10-c)(0.01) \quad p(3)=0.30$$
$$p(4)=0.33 \quad p(5)=0.33+c(0.01) \quad p(6)=0.5-(10-c)(0.01)$$

Donde $p(n)$ es la probabilidad de cazar a una presa en un grupo de $n$ leones y $c$ es el último número de su código Uniandes.

a) Formule este problema como un proceso de decisión de Markov (MDP). Es decir, defina estados, acciones y recompensas. Asuma una única caza por día y que sí la caza es exitosa todos los leones se dividen la carne en partes iguales. Tenga en cuenta que el objetivo del león es maxizimizar su probabilidad de supervivencia sobre $T$ días.

## Solución
$$\text {Variable de estado }  \\ X_n: \text {Cantidad de alimento (en kg) en el estomago de un león después de la n-ésima caza} $$
$$\text {Espacio de estados } \\S : \{0,0.5,1,...,30\}  $$
Por fines prácticos se decide que el espacio de estados estará discretizado en pasos de 0.5 kg
$$\text {Acciones}: \{0,1,2,3,4,5,6\}$$
Donde 0 significa que el león decide no cazar, 1 que decide cazar individualmente, 2,3,4,5,6 que caza en grupo de tamaño 2,3,4,5 y 6 respectivamente.
$$\text {Recompensas}: $$
    $$ r_{t}(s,a)  = 
    \begin{cases} 
    -1, & s=0 \\
    0, & \text{d.l.c}
    \end{cases}
    $$

Ahora bien, la forma de calcular las matrices de probabilidad de transición, es la siguiente:
$$ p(i,j|a)= 
    \begin{cases}
    1, & j=i-12 & i>12 & a=0 \\
    1, & j=0 & i\leq12 & a=0 \\
    1-p(a), & j=i-13 & i>13 & i>0 & a\neq0 \\
    1-p(a), & j=0 & i\leq 13 & i>0 & a\neq0 \\
    p(a), & j=60 & i+ \lfloor \frac{164\cdot2}{a}\rfloor -13 >60 & i>0 & a\neq0 \\
    p(a), & j=i+ \lfloor \frac{164\cdot2}{a}\rfloor -13 & i>0 & a\neq0 \\
    1, & i=0 & j=0 & a\neq0 \\
    0, & d.l.c
    \end {cases}
    $$

In [3]:
# Importar librerias
import numpy as np

#Definir el conjunto de estados
estados = np.arange(61)
estados
#Definir el conjunto de acciones 
acciones = np.arange(7)
acciones 
recompensas = np.zeros(len(estados))
recompensas[0] = -1 
# Parámetro del último código de alguno de los dos miembros del grupo
c = int(input("Ingrese el último digito de su código uniandes: ")) 

#Crear función para calcular las probabilidades fácilmente
def prob(a):
    if a==1:
        resultado = 0.1 + (c*0.01)
    if a==2:
        resultado = 0.27 - (10-c)*(0.01)
    if a==3:
        resultado = 0.3
    if a==4:
        resultado = 0.33
    if a==5:
        resultado = 0.35 + (c*0.01)
    if a==6:
        resultado = 0.5 - (10-c)*(0.01)
    return resultado


Ingrese el último digito de su código uniandes: 4


In [4]:
# Crear las matrices de transición de probabilidad, una matriz por cada acción
P = np.zeros((len(estados),len(estados),len(acciones)))

In [5]:
# Llenar las matrices de transición de probabilidad
for a in acciones:
    for i in estados:
        for j in estados:
            if a==0:
                # Consume de lo que tiene almacenado en el estomago
                if (j==i-12) & (i>12):
                    P[i,j,a] = 1
                # Se queda sin reservas
                if (j==0) & (i<=12):
                    P[i,j,a] = 1
            else:
                # Caza fallida 
                if (j==i-13) & (i>0):
                    P[i,j,a] = 1-prob(a)
                #Caza exitosa 
                if (j==60) & (i+np.rint((164*2)/a)-13>60) & (i>0):
                    P[i,j,a] = prob(a)
                #Caza exitosa 
                if (j==i-13+np.rint((164*2)/a)) & (i>0):
                    P[i,j,a] = prob(a)
                # Caza fallida y agota las reservas
                if (j==0) & (i<=13)  & (i>0):
                    P[i,j,a] = 1-prob(a)
                # Estado absorbente (está muerto no tiene energía ni siquiera para cazar)
                if (j==0) & (i==0):
                    P[i,j,a] =1

#Comprobar que la matriz sea estocastica
for i in acciones:
    print(i,P[:,:,i].sum(axis=1),sep="\n")


0
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
1
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
2
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
3
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
4
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
5
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1

## Value iteration 

Para hallar una solución óptima al problema del Lazy Adaptable Lion, se empezó por implementar el algoritmo de iteración de valor. 

In [6]:
%%time

#Crear un vector para guardar la función de valor para cada estado
V = np.zeros(len(estados))

# Crear un vector para guardar la política óptima
Pi = np.zeros(len(estados))

# Factor de descuento
gamma = 0.9

# Contador
k = 1

error = 1
while error > 0.001:
    error = 0
    
    for s in estados:
        # Inicializar la funcion de valor de cada acción para el estado s
        q = np.zeros(len(acciones))
        temp = V[s]*1.0
        
        # Calcular la función de valor para cada acción
        for a in acciones:
            if s!=0:
                q[a] = sum([P[s, s_prima, a]*(recompensas[s]+gamma*V[s_prima]) for s_prima in estados])
            else:
                q[a] =  sum([P[s, s_prima, a]*(recompensas[s]) for s_prima in estados])
        
        # Obtener la función de valor V maximizando q
        V[s] = max(q)
        # Obtener el argumento (acción) que maximiza q
        Pi[s] = np.argmax(q)
        
        # Determinar si la función de valor ha convergido
        error = max(error, abs(temp-V[s]))
        
    k = k+1
    print("Iteración:",k," Error:", error)  

Iteración: 2  Error: 1.0
Iteración: 3  Error: 0.025937084837618057
Iteración: 4  Error: 0.020033743101755846
Iteración: 5  Error: 0.01542871972644537
Iteración: 6  Error: 0.01190331343775751
Iteración: 7  Error: 0.00918465076173873
Iteración: 8  Error: 0.007086916003957999
Iteración: 9  Error: 0.005468294650203209
Iteración: 10  Error: 0.004219359501572262
Iteración: 11  Error: 0.0032556758082130377
Iteración: 12  Error: 0.0025120933554257097
Iteración: 13  Error: 0.0019383419597379659
Iteración: 14  Error: 0.001495632932894786
Iteración: 15  Error: 0.0011540367574057464
Iteración: 16  Error: 0.0008904596897754968
CPU times: user 291 ms, sys: 5.22 ms, total: 296 ms
Wall time: 292 ms


A continuación se puede observar la función de valor para cada estado:

In [7]:
print(" ", "Estado", "V")
for s in estados:
    print(s, " ", s/2, "kg", " ",round(V[s], 4), sep="")

  Estado V
0 0.0kg -1.0
1 0.5kg -0.5718
2 1.0kg -0.5718
3 1.5kg -0.5718
4 2.0kg -0.5718
5 2.5kg -0.5718
6 3.0kg -0.5718
7 3.5kg -0.5715
8 4.0kg -0.5712
9 4.5kg -0.5712
10 5.0kg -0.565
11 5.5kg -0.5603
12 6.0kg -0.5603
13 6.5kg -0.5146
14 7.0kg -0.3445
15 7.5kg -0.3445
16 8.0kg -0.3445
17 8.5kg -0.3445
18 9.0kg -0.3445
19 9.5kg -0.3445
20 10.0kg -0.3443
21 10.5kg -0.3442
22 11.0kg -0.3442
23 11.5kg -0.3411
24 12.0kg -0.3387
25 12.5kg -0.3387
26 13.0kg -0.31
27 13.5kg -0.2299
28 14.0kg -0.2299
29 14.5kg -0.2299
30 15.0kg -0.2299
31 15.5kg -0.2299
32 16.0kg -0.2299
33 16.5kg -0.2298
34 17.0kg -0.2297
35 17.5kg -0.2297
36 18.0kg -0.2282
37 18.5kg -0.227
38 19.0kg -0.227
39 19.5kg -0.2069
40 20.0kg -0.1721
41 20.5kg -0.1721
42 21.0kg -0.1721
43 21.5kg -0.1721
44 22.0kg -0.1721
45 22.5kg -0.1721
46 23.0kg -0.1721
47 23.5kg -0.1721
48 24.0kg -0.1721
49 24.5kg -0.1713
50 25.0kg -0.1707
51 25.5kg -0.1707
52 26.0kg -0.1549
53 26.5kg -0.143
54 27.0kg -0.143
55 27.5kg -0.143
56 28.0kg -0.143
57 28

A continuación se puede observar la acción a tomar en cada estado:

In [8]:
for s in estados:
    print('Acción a tomar en', s/2, "kg:", Pi[s])

Acción a tomar en 0.0 kg: 0.0
Acción a tomar en 0.5 kg: 6.0
Acción a tomar en 1.0 kg: 6.0
Acción a tomar en 1.5 kg: 6.0
Acción a tomar en 2.0 kg: 6.0
Acción a tomar en 2.5 kg: 6.0
Acción a tomar en 3.0 kg: 6.0
Acción a tomar en 3.5 kg: 6.0
Acción a tomar en 4.0 kg: 6.0
Acción a tomar en 4.5 kg: 6.0
Acción a tomar en 5.0 kg: 6.0
Acción a tomar en 5.5 kg: 6.0
Acción a tomar en 6.0 kg: 6.0
Acción a tomar en 6.5 kg: 0.0
Acción a tomar en 7.0 kg: 6.0
Acción a tomar en 7.5 kg: 6.0
Acción a tomar en 8.0 kg: 6.0
Acción a tomar en 8.5 kg: 6.0
Acción a tomar en 9.0 kg: 6.0
Acción a tomar en 9.5 kg: 6.0
Acción a tomar en 10.0 kg: 6.0
Acción a tomar en 10.5 kg: 6.0
Acción a tomar en 11.0 kg: 6.0
Acción a tomar en 11.5 kg: 6.0
Acción a tomar en 12.0 kg: 6.0
Acción a tomar en 12.5 kg: 6.0
Acción a tomar en 13.0 kg: 0.0
Acción a tomar en 13.5 kg: 6.0
Acción a tomar en 14.0 kg: 6.0
Acción a tomar en 14.5 kg: 6.0
Acción a tomar en 15.0 kg: 6.0
Acción a tomar en 15.5 kg: 6.0
Acción a tomar en 16.0 kg: 6

## Policy iteration

Luego, se implementó el algoritmo de iteración de política. La estructura del algoritmo y si implementación se observan a continuación.

### Algoritmo

> Inicializar $\pi_{0}(s)$ de forma arbitraria<br>
> Inicializar $V_{0}(s)$ y $Q_{0}(s,a)$ en 0.0 para todos los estados y acciones<br>
> policy_stable = False<br>
> Mientras(not policy_stable):<br>
>> 1. Evaluar la política<br>
>> Para $k=1,\cdots,H$:<br>
>>> Para todos los estados $s$:<br>
>>>> Para todas las acciones $s$ permitidas en $s$:<br>
>>>>> $$Q_{k}(s,a)=\sum_{s'} P(s'|s,\pi(s))(R(s,a,s')+\gamma Q_{k-1}(s',\pi(s')))$$<br>
>>>> Valor del estado $s$ corresponde al valor Q de la acción a ejecutar según la política $\pi_k(s)$ <br>
>>>> $V_{k}(s)=Q_{k}(s,\pi_{k}(s))$ <br>
>
>> 2. Mejorar la política <br>
>> policy_stable = True <br>
>> Para todos los estados $s$:<br>
>>> best_action = $arg\max_{a}Q_k(s,)$<br>
>>> si *best_action* es diferente a lo que dice $\pi_{k}(s)$:<br>
>>>> $\pi_{k}(s) = best\_action$<br>
>>>> policy_stable = False<br>

Se empezó por crear una función encargada de realizar la evaluación de la política.

In [9]:
# Evaluar la política

def evaluar_politica():
    error = 1
    while error > 0.001:
        error = 0
        for s in estados:
            q = np.zeros(len(acciones))
            temp = V[s]*1.0

            for a in acciones:
                if s!=0:
                    q[a] = sum([P[s, s_prima, a]*(recompensas[s]+gamma*Q[s_prima, pi_base[s_prima]]) for s_prima in estados])
                else:
                    q[a] =  sum([P[s, s_prima, a]*(recompensas[s]) for s_prima in estados])
            
            Q[s,:] = q.copy()

            V[s] = Q[s, pi_base[s]]

            error = max(error, abs(temp-V[s]))

Luego, se inicializa una política arbitraria que en este caso consta de nunca cazar. También se inicializa en 0 la función de valor y la función de pares estado-acción.

In [10]:
pi_base = np.zeros(len(estados)).astype(int)
gamma = 0.9
Q = np.zeros((len(estados), len(acciones)))
V = np.zeros(len(estados))

Finalmente, se implementa todo el algoritmo, mejorando la política seleccionando la acción greedy hasta que la política converge.

In [11]:
%%time

policy_stable = False
c = 1
while(not policy_stable):
    
    # Evaluar la política actual
    evaluar_politica()
    
    policy_stable = True

    for s in estados:
        
        # Seleccionar la acción greedy
        best_action = np.argmax(Q[s,:])
        
        # Verificar si la acción greedy es igual a la acción definida por la política actual
        if best_action != pi_base[s]:
            pi_base[s] = best_action
            policy_stable = False
    
    print("fin iteracion", c)
    c = c + 1

fin iteracion 1
fin iteracion 2
fin iteracion 3
CPU times: user 504 ms, sys: 8.47 ms, total: 513 ms
Wall time: 505 ms


A continuación se puede observar la función de valor para cada estado:

In [12]:
print(" ", "Estado", "V")
for s in estados:
    print(s, " ", s/2, "kg", " ",round(V[s], 4), sep="")

  Estado V
0 0.0kg -1.0
1 0.5kg -0.574
2 1.0kg -0.574
3 1.5kg -0.574
4 2.0kg -0.574
5 2.5kg -0.574
6 3.0kg -0.574
7 3.5kg -0.5737
8 4.0kg -0.5734
9 4.5kg -0.5734
10 5.0kg -0.567
11 5.5kg -0.5626
12 6.0kg -0.5625
13 6.5kg -0.5166
14 7.0kg -0.3478
15 7.5kg -0.3478
16 8.0kg -0.3478
17 8.5kg -0.3478
18 9.0kg -0.3478
19 9.5kg -0.3478
20 10.0kg -0.3477
21 10.5kg -0.3475
22 11.0kg -0.3475
23 11.5kg -0.3443
24 12.0kg -0.3421
25 12.5kg -0.3421
26 13.0kg -0.3131
27 13.5kg -0.2338
28 14.0kg -0.2338
29 14.5kg -0.2338
30 15.0kg -0.2338
31 15.5kg -0.2338
32 16.0kg -0.2338
33 16.5kg -0.2338
34 17.0kg -0.2337
35 17.5kg -0.2337
36 18.0kg -0.2321
37 18.5kg -0.2309
38 19.0kg -0.2309
39 19.5kg -0.2105
40 20.0kg -0.1764
41 20.5kg -0.1764
42 21.0kg -0.1764
43 21.5kg -0.1764
44 22.0kg -0.1764
45 22.5kg -0.1764
46 23.0kg -0.1763
47 23.5kg -0.1763
48 24.0kg -0.1763
49 24.5kg -0.1755
50 25.0kg -0.1749
51 25.5kg -0.1749
52 26.0kg -0.1588
53 26.5kg -0.1474
54 27.0kg -0.1474
55 27.5kg -0.1474
56 28.0kg -0.1474
57 

A continuación se puede observar la acción a tomar en cada estado:

In [13]:
for s in estados:
    print('Acción a tomar en', s/2, "kg:", pi_base[s])

Acción a tomar en 0.0 kg: 0
Acción a tomar en 0.5 kg: 6
Acción a tomar en 1.0 kg: 6
Acción a tomar en 1.5 kg: 6
Acción a tomar en 2.0 kg: 6
Acción a tomar en 2.5 kg: 6
Acción a tomar en 3.0 kg: 6
Acción a tomar en 3.5 kg: 6
Acción a tomar en 4.0 kg: 6
Acción a tomar en 4.5 kg: 6
Acción a tomar en 5.0 kg: 6
Acción a tomar en 5.5 kg: 6
Acción a tomar en 6.0 kg: 6
Acción a tomar en 6.5 kg: 0
Acción a tomar en 7.0 kg: 6
Acción a tomar en 7.5 kg: 6
Acción a tomar en 8.0 kg: 6
Acción a tomar en 8.5 kg: 6
Acción a tomar en 9.0 kg: 6
Acción a tomar en 9.5 kg: 6
Acción a tomar en 10.0 kg: 6
Acción a tomar en 10.5 kg: 6
Acción a tomar en 11.0 kg: 6
Acción a tomar en 11.5 kg: 6
Acción a tomar en 12.0 kg: 6
Acción a tomar en 12.5 kg: 6
Acción a tomar en 13.0 kg: 0
Acción a tomar en 13.5 kg: 6
Acción a tomar en 14.0 kg: 6
Acción a tomar en 14.5 kg: 6
Acción a tomar en 15.0 kg: 6
Acción a tomar en 15.5 kg: 6
Acción a tomar en 16.0 kg: 6
Acción a tomar en 16.5 kg: 6
Acción a tomar en 17.0 kg: 6
Acció

## Conclusiones

- Se puede observar que para ambos algorítmos, se obtienen políticas similares donde en la gran mayoría de los estados se determina que la acción óptima a tomar es la de cazar en un grupo de 6 leones. Aunque las dos políticas no son identicas, las funciones de valor $V$ para ambas políticas son consistentes entre si, por lo que podemos concluir que ambas políticas son óptimas y ambos algoritmos son efectivos.

- Se pudo observar que el algoritmo de Value Iteration es más eficiente debido a que tuvo un tiempo de ejecución equivalente al 58% del tiempo de ejecución de Policy iteration. 